# Nueral Network for Predicting World Series Champions
The procedure has been borrowed from the machine learning class examples

# Read in data and pre-process

In [1]:
# import dependencies
import pandas as pd

In [2]:
# read in data
pd.set_option('display.max_columns', None)
mlb_df = pd.read_csv('resources/mlb_data.csv')
# mlb_df.head()

In [3]:
# change the League column to be binary instead of string
for i, row in mlb_df.iterrows():
#     print(i,row['Lg'])
    if row['Lg'] == 'AL':
        mlb_df.at[i,'Lg'] = 0
#         print(f'new value at {i} is {mlb_df.at[i,"Lg"]}')
    elif row['Lg'] == 'NL':
        mlb_df.at[i,'Lg'] = 1
#         print(f'new value at {i} is {mlb_df.at[i,"Lg"]}')
    

In [4]:
# league is now a 1 or 0
# mlb_df.head()
# mlb_df.dtypes

# Split into training and testing set
Manually split the data by season. Odd years will be used for training, while even years will be used for testing

In [5]:
# get X and Y train for making the model
train_df = mlb_df.loc[mlb_df['Year'] % 2 == 1]
train_data = train_df.values
X_train = train_data[:, 3:22]
y_train = train_data[:,23]

In [6]:
# get X and Y for making the model
test_df = mlb_df.loc[mlb_df['Year'] % 2 == 0]
test_data = test_df.values
X_test = test_data[:, 3:22]
y_test = test_data[:,23]

# Scaling and One-hot encoding

In [7]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)

In [8]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# label encode the winner column
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
label_encoder.fit(y_test)
encoded_y_test = label_encoder.transform(y_test)

In [10]:
# One-hot encoding
from keras.utils import to_categorical

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)
# y_train_categorical

Using TensorFlow backend.
C:\Users\Owner\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Owner\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Owner\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Owner\anaconda3\envs\Py

# Create model

In [11]:
from tensorflow.keras.models import Sequential

model = Sequential()

In [12]:
X_train.shape

(300, 19)

In [13]:
from tensorflow.keras.layers import Dense
number_inputs = 19
number_hidden_nodes = 10
model.add(Dense(units=number_hidden_nodes, activation='relu', input_dim=number_inputs))
model.add(Dense(10, activation='relu'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
number_classes = 2
model.add(Dense(units=number_classes, activation='softmax'))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                200       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 22        
Total params: 332
Trainable params: 332
Non-trainable params: 0
_________________________________________________________________


In [16]:
# # Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [17]:
print(X_train_scaled.shape)
print(y_train_categorical.shape)

(300, 19)
(300, 2)


In [18]:
# Fit the model
model.fit(
    x=X_train_scaled,
    y=y_train_categorical,
    epochs=1000,
    shuffle=True,
    verbose=2
)

Epoch 1/1000
300/300 - 0s - loss: 0.6243 - acc: 0.6633
Epoch 2/1000
300/300 - 0s - loss: 0.5386 - acc: 0.8067
Epoch 3/1000
300/300 - 0s - loss: 0.4680 - acc: 0.9100
Epoch 4/1000
300/300 - 0s - loss: 0.4061 - acc: 0.9600
Epoch 5/1000
300/300 - 0s - loss: 0.3526 - acc: 0.9667
Epoch 6/1000
300/300 - 0s - loss: 0.3100 - acc: 0.9667
Epoch 7/1000
300/300 - 0s - loss: 0.2737 - acc: 0.9667
Epoch 8/1000
300/300 - 0s - loss: 0.2448 - acc: 0.9667
Epoch 9/1000
300/300 - 0s - loss: 0.2215 - acc: 0.9667
Epoch 10/1000
300/300 - 0s - loss: 0.2027 - acc: 0.9667
Epoch 11/1000
300/300 - 0s - loss: 0.1880 - acc: 0.9667
Epoch 12/1000
300/300 - 0s - loss: 0.1760 - acc: 0.9667
Epoch 13/1000
300/300 - 0s - loss: 0.1667 - acc: 0.9667
Epoch 14/1000
300/300 - 0s - loss: 0.1583 - acc: 0.9667
Epoch 15/1000
300/300 - 0s - loss: 0.1512 - acc: 0.9667
Epoch 16/1000
300/300 - 0s - loss: 0.1452 - acc: 0.9667
Epoch 17/1000
300/300 - 0s - loss: 0.1400 - acc: 0.9667
Epoch 18/1000
300/300 - 0s - loss: 0.1355 - acc: 0.9667
E

Epoch 147/1000
300/300 - 0s - loss: 0.0255 - acc: 0.9967
Epoch 148/1000
300/300 - 0s - loss: 0.0253 - acc: 0.9967
Epoch 149/1000
300/300 - 0s - loss: 0.0253 - acc: 0.9967
Epoch 150/1000
300/300 - 0s - loss: 0.0252 - acc: 0.9967
Epoch 151/1000
300/300 - 0s - loss: 0.0251 - acc: 0.9967
Epoch 152/1000
300/300 - 0s - loss: 0.0250 - acc: 0.9967
Epoch 153/1000
300/300 - 0s - loss: 0.0249 - acc: 0.9967
Epoch 154/1000
300/300 - 0s - loss: 0.0247 - acc: 0.9967
Epoch 155/1000
300/300 - 0s - loss: 0.0247 - acc: 0.9967
Epoch 156/1000
300/300 - 0s - loss: 0.0246 - acc: 0.9967
Epoch 157/1000
300/300 - 0s - loss: 0.0245 - acc: 0.9967
Epoch 158/1000
300/300 - 0s - loss: 0.0244 - acc: 0.9967
Epoch 159/1000
300/300 - 0s - loss: 0.0243 - acc: 0.9967
Epoch 160/1000
300/300 - 0s - loss: 0.0242 - acc: 0.9967
Epoch 161/1000
300/300 - 0s - loss: 0.0241 - acc: 0.9967
Epoch 162/1000
300/300 - 0s - loss: 0.0240 - acc: 0.9967
Epoch 163/1000
300/300 - 0s - loss: 0.0239 - acc: 0.9967
Epoch 164/1000
300/300 - 0s - l

Epoch 291/1000
300/300 - 0s - loss: 0.0159 - acc: 0.9967
Epoch 292/1000
300/300 - 0s - loss: 0.0159 - acc: 0.9967
Epoch 293/1000
300/300 - 0s - loss: 0.0159 - acc: 0.9967
Epoch 294/1000
300/300 - 0s - loss: 0.0158 - acc: 0.9967
Epoch 295/1000
300/300 - 0s - loss: 0.0158 - acc: 0.9967
Epoch 296/1000
300/300 - 0s - loss: 0.0158 - acc: 0.9967
Epoch 297/1000
300/300 - 0s - loss: 0.0157 - acc: 0.9967
Epoch 298/1000
300/300 - 0s - loss: 0.0157 - acc: 0.9967
Epoch 299/1000
300/300 - 0s - loss: 0.0156 - acc: 0.9967
Epoch 300/1000
300/300 - 0s - loss: 0.0156 - acc: 0.9967
Epoch 301/1000
300/300 - 0s - loss: 0.0155 - acc: 0.9967
Epoch 302/1000
300/300 - 0s - loss: 0.0155 - acc: 0.9967
Epoch 303/1000
300/300 - 0s - loss: 0.0155 - acc: 0.9967
Epoch 304/1000
300/300 - 0s - loss: 0.0154 - acc: 0.9967
Epoch 305/1000
300/300 - 0s - loss: 0.0154 - acc: 0.9967
Epoch 306/1000
300/300 - 0s - loss: 0.0154 - acc: 0.9967
Epoch 307/1000
300/300 - 0s - loss: 0.0153 - acc: 0.9967
Epoch 308/1000
300/300 - 0s - l

Epoch 435/1000
300/300 - 0s - loss: 0.0124 - acc: 0.9967
Epoch 436/1000
300/300 - 0s - loss: 0.0124 - acc: 0.9967
Epoch 437/1000
300/300 - 0s - loss: 0.0124 - acc: 0.9967
Epoch 438/1000
300/300 - 0s - loss: 0.0123 - acc: 0.9967
Epoch 439/1000
300/300 - 0s - loss: 0.0123 - acc: 0.9967
Epoch 440/1000
300/300 - 0s - loss: 0.0123 - acc: 0.9967
Epoch 441/1000
300/300 - 0s - loss: 0.0123 - acc: 0.9967
Epoch 442/1000
300/300 - 0s - loss: 0.0123 - acc: 0.9967
Epoch 443/1000
300/300 - 0s - loss: 0.0123 - acc: 0.9967
Epoch 444/1000
300/300 - 0s - loss: 0.0123 - acc: 0.9967
Epoch 445/1000
300/300 - 0s - loss: 0.0122 - acc: 0.9967
Epoch 446/1000
300/300 - 0s - loss: 0.0122 - acc: 0.9967
Epoch 447/1000
300/300 - 0s - loss: 0.0122 - acc: 0.9967
Epoch 448/1000
300/300 - 0s - loss: 0.0122 - acc: 0.9967
Epoch 449/1000
300/300 - 0s - loss: 0.0122 - acc: 0.9967
Epoch 450/1000
300/300 - 0s - loss: 0.0122 - acc: 0.9967
Epoch 451/1000
300/300 - 0s - loss: 0.0122 - acc: 0.9967
Epoch 452/1000
300/300 - 0s - l

Epoch 579/1000
300/300 - 0s - loss: 0.0114 - acc: 0.9967
Epoch 580/1000
300/300 - 0s - loss: 0.0114 - acc: 0.9967
Epoch 581/1000
300/300 - 0s - loss: 0.0114 - acc: 0.9967
Epoch 582/1000
300/300 - 0s - loss: 0.0114 - acc: 0.9967
Epoch 583/1000
300/300 - 0s - loss: 0.0114 - acc: 0.9967
Epoch 584/1000
300/300 - 0s - loss: 0.0114 - acc: 0.9967
Epoch 585/1000
300/300 - 0s - loss: 0.0114 - acc: 0.9967
Epoch 586/1000
300/300 - 0s - loss: 0.0114 - acc: 0.9967
Epoch 587/1000
300/300 - 0s - loss: 0.0114 - acc: 0.9967
Epoch 588/1000
300/300 - 0s - loss: 0.0114 - acc: 0.9967
Epoch 589/1000
300/300 - 0s - loss: 0.0114 - acc: 0.9967
Epoch 590/1000
300/300 - 0s - loss: 0.0114 - acc: 0.9967
Epoch 591/1000
300/300 - 0s - loss: 0.0114 - acc: 0.9967
Epoch 592/1000
300/300 - 0s - loss: 0.0114 - acc: 0.9967
Epoch 593/1000
300/300 - 0s - loss: 0.0114 - acc: 0.9967
Epoch 594/1000
300/300 - 0s - loss: 0.0114 - acc: 0.9967
Epoch 595/1000
300/300 - 0s - loss: 0.0114 - acc: 0.9967
Epoch 596/1000
300/300 - 0s - l

Epoch 723/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 724/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 725/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 726/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 727/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 728/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 729/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 730/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 731/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 732/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 733/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 734/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 735/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 736/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 737/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 738/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 739/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 740/1000
300/300 - 0s - l

Epoch 867/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 868/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 869/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 870/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 871/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 872/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 873/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 874/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 875/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 876/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 877/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 878/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 879/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 880/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 881/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 882/1000
300/300 - 0s - loss: 0.0112 - acc: 0.9967
Epoch 883/1000
300/300 - 0s - loss: 0.0093 - acc: 0.9967
Epoch 884/1000
300/300 - 0s - l